In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtChart import *
from PyQt5 import uic
from PyQt5 import QtChart as qc
import win32com.client
import ctypes
import pandas as pd
import os
import time
import datetime
import numpy as np
import threading
import sys
from pywinauto import application
from pywinauto import timings
import csv

import nbimporter
import autologin
import cybos_trade
import cybos_watch
import cybos_inquiry
import cybos_current

Importing Jupyter notebook from autologin.ipynb
Importing Jupyter notebook from cybos_trade.ipynb
Importing Jupyter notebook from cybos_watch.ipynb
Importing Jupyter notebook from cybos_inquiry.ipynb
Importing Jupyter notebook from cybos_current.ipynb


In [4]:
login = autologin.cp_login() # 일반 로그인
login.login_virtual()

공지사항 창 닫힘.


In [5]:
# API 모듈 import
obj_codeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr') # 종목정보
obj_cpStatus = win32com.client.Dispatch('CpUtil.CpCybos') # 연결, 상태 정보
obj_cpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil') # 거래 초기화
obj_cpTd0311 = win32com.client.Dispatch('CpTrade.CpTd0311') # 거래 오브젝트
obj_stockMst = win32com.client.Dispatch("DsCbo1.StockMst") # 현재가
obj_stockChart = win32com.client.Dispatch("CpSysDib.StockChart") # 차트 데이터

In [6]:
# 상태 체크
def init_pluscheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('Cybos Plus - 정상 (관리자권한으로 실행된 프로세스입니다.)')
    else:
        print('Cybos Plus - 오류 (일반권한으로 실행됨. 관리자 권한으로 실행해 주세요)')
        return False
 
    # 연결 여부 체크
    if (obj_cpStatus.IsConnect == 0):
        print("Cybos Plus - PLUS 가 정상적으로 연결되지 않음. ")
        return False
    
    # 주문 관련 초기화
    if (obj_cpTrade.TradeInit(0) != 0):
        print("Cybos Plus - 주문 초기화 실패")
        return False
    else:
        print('Cybos Plus - 주문 초기화 완료')
    return True

init_pluscheck() # 초기화

Cybos Plus - 정상 (관리자권한으로 실행된 프로세스입니다.)
Cybos Plus - 주문 초기화 완료


True

In [7]:
form = uic.loadUiType('C:\\Users\\김민수\\Desktop\\trading_tmp\\Trader_watch.ui')[0] # UI 로드

class myWindow(QMainWindow, form):
    def __init__(self):
        super().__init__()
        self.setupUi(self) # UI 셋팅
        self.set_timer_clock() # 시간 실시간 업뎃
        self.set_timer_collective() # 루프 일괄 실시간 업뎃
        self.set_tables() # 테이블 셋팅
        self.edit_plus.setText('2') # 익절기준 초기값 3
        self.edit_minus.setText('3') # 손절기준 초기값 -3
        
        # 모듈 로드
        self.obj_watch = cybos_watch.Watch(obj_codeMgr) # 전략 감시 모듈
        self.obj_trade = cybos_trade.Trade(obj_cpTrade.AccountNumber[0], self) # 매매 거래 모듈
        self.obj_inquiry = cybos_inquiry.Inquiry() # 잔고 및 미체결 조회 모듈

        # -- UI 이벤트 콜백함수 등록 --
        #self.btn_watch.clicked.connect(self.btn_watch_clicked) # 감시 시작/중지
        
        self.is_watching = False # 감시 중 여부
        self.max_item = 10 # 최대 보유 종목 제한
        self.history = [] # 매도 내역 - csv 로 저장용

        # ----- 테스트
    
    # -----------
    # - 이벤트 함수  
    # ---------
    
    # 감시 시작 / 중지
    def btn_watch_clicked(self):
        # 감시 시작
        if(not self.is_watching):
            self.is_watching = True
            self.btn_watch.setText('감시 중지')
            self.obj_watch.listAllStrategy()
            self.obj_watch.watch_stg()
        # 감시 중지
        else:
            self.obj_watch.btnExit_clicked()
            self.is_watching = False
            self.btn_watch.setText('감시 시작')

    
    # -----------
    # - 유틸 
    # ---------    
    
    # 시간 표시 함수
    def set_timer_clock(self):
        self.timer_date = QTimer()
        self.timer_date.timeout.connect(self.update_time) # 시간 업데이트 함수 연결
        self.timer_date.setSingleShot(False)
        self.timer_date.start(1000)
        
    def update_time(self):
        self.label_clock.setText(QDateTime.currentDateTime().toString(Qt.DefaultLocaleLongDate)) # 현재 시간 표시

    # 주식 계좌 조회 전용 타이머 - 3 초 주기
    def set_timer_collective(self):
        self.timer_account = QTimer()
        self.timer_account.timeout.connect(self.collecive_update) # 잔고 조회 함수 연결
        self.timer_account.setSingleShot(False)
        self.timer_account.start(2000)
        
    # table 셋팅
    def set_tables(self):
        # 잔고 데이터 테이블
        self.tableWidget_jango.setColumnCount(7)
        self.tableWidget_jango.setHorizontalHeaderLabels(('종목명','현재가','수익률','장부가','보유물량','매도가능','평가 손익'))
        # cell 너비 조정
        self.tableWidget_jango.setColumnWidth(0, 150)
        self.tableWidget_jango.setColumnWidth(1, 50)
        self.tableWidget_jango.setColumnWidth(2, 50)
        self.tableWidget_jango.setColumnWidth(3, 50)
        self.tableWidget_jango.setColumnWidth(4, 100)
        self.tableWidget_jango.setColumnWidth(5, 100)
        self.tableWidget_jango.setColumnWidth(6, 100)
        
        # 미체결 데이터 테이블
        self.tableWidget_miss.setColumnCount(5)
        self.tableWidget_miss.setHorizontalHeaderLabels(('종목명','주문수량','체결수량','주문구분','주문단가'))
        
        # 감시 데이터 테이블
        self.tableWidget_watch.setColumnCount(6)
        self.tableWidget_watch.setHorizontalHeaderLabels(('종목명','현재가','거래량','거래대금','매도호가','매수호가'))
        
        # 내역 데이터 테이블
        self.tableWidget_history.setColumnCount(3)
        self.tableWidget_history.setHorizontalHeaderLabels(('종목명','수익률','평가손익'))
    
            
    # 일괄적으로 갱신 - 잔고, 미체결, 주문, 감시 리스트
    def collecive_update(self):
        
         # 황금시간대에만 거래 할것임. -----------
        now = datetime.datetime.now()
        if(now.hour >= 11):
            self.quit()
        elif(now.hour >= 10):
            pass
        elif(now.hour >= 9 and now.minute >= 30):
            pass
        else:
            return
        
        # 데이터 갱신
        self.jango_data, cur, self.total_data = self.obj_inquiry.get_accountdata() # 주식 잔고 데이터, 실시간 수신하고있는 데이터 접근
        self.obj_inquiry.Reqeust5339() # 미체결 주문 요청
        self.obj_watch.watch_once() # 감시 데이터 요청
        
        # 잔고
        try:
            self.jango_update()
        except:
            print('예외 : 잔고')
        
        # 미체결
        try:
            self.miss_update()
        except:
            print('예외 : 미체결')

        # 총 평가금 표시
        total_eval = self.obj_inquiry.total_data['예상예수금']
        self.label_eval.setText('평가금액 : {}'.format(format(total_eval,',')))
                
        # 맨 앞으로 띄우기
        #window = application.Application().connect(title_re='Trader_v2')
        #window.Trader_v2.set_focus()
        
        # 감시
        try:
            self.watch_update()
        except:
            print('예외 : 감시')
            
        # 자동 매수
        try:
            self.auto_buy()
        except:
            print('예외 : 매수')
        
        # 자동 매매
        try:
            self.auto_sell()
        except:
            print('예외 : 매도')
        
        
    # 잔고 데이터 업데이트 / table 에 그리기
    def jango_update(self):
        self.tableWidget_jango.setRowCount(0) # 행 갯수 0 으로 초기화
        
        for data in self.jango_data.items():
            item_list = []
            item_list.append(QTableWidgetItem(data[1]['종목명'])) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(data[1]['현재가'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(str(round(data[1]['수익률'],2))+'%')) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(int(data[1]['장부가']),','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(str(data[1]['잔고수량']))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(str(data[1]['매도가능']))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(int(data[1]['평가손익']/1000),','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            
            # 아이템 텍스트 정렬
            for idx in range(1, len(item_list)):
                item_list[idx].setTextAlignment(Qt.AlignRight | Qt.AlignVCenter)
            
            rowPosition = self.tableWidget_jango.rowCount()
            self.tableWidget_jango.insertRow(rowPosition)
            
            # 아이템 추가
            for idx in range(len(item_list)):
                self.tableWidget_jango.setItem(rowPosition, idx, item_list[idx])
            
            # 차트 cell, text 색깔
            if(data[1]['수익률'] > 0):
                color = Qt.red
                color_txt = Qt.green
            else:
                color = Qt.blue
                color_txt = Qt.yellow
                
            for j in range(self.tableWidget_jango.columnCount()):
                self.tableWidget_jango.item(rowPosition, j).setBackground(color)
                self.tableWidget_jango.item(rowPosition, j).setForeground(color_txt)
            
            self.tableWidget_jango.setRowHeight(rowPosition, 20) # cell 높이 설정
        
        
    # 미체결 주문 데이터 업데이트 / table 에 그리기
    def miss_update(self):
        self.tableWidget_miss.setRowCount(0) # 행 갯수 0 으로 초기화
        
        for data in self.obj_inquiry.diOrderList.values():
            item_list = []
            item_list.append(QTableWidgetItem(data.name)) # 종목명
            item_list.append(QTableWidgetItem(str(data.amount))) # 주문수량
            item_list.append(QTableWidgetItem(str(data.ContAmount))) # 체결 수량
            item_list.append(QTableWidgetItem('매매' if data.buysell==1 else '매수')) # 매매구분
            item_list.append(QTableWidgetItem(format(data.price,','))) # 주문단가
            
            # 아이템 텍스트 정렬
            for idx in range(1, len(item_list)):
                item_list[idx].setTextAlignment(Qt.AlignRight | Qt.AlignVCenter)
            
            rowPosition = self.tableWidget_miss.rowCount()
            self.tableWidget_miss.insertRow(rowPosition)
            
            # 아이템 추가
            for idx in range(len(item_list)):
                self.tableWidget_miss.setItem(rowPosition, idx, item_list[idx])
            
            self.tableWidget_miss.setRowHeight(rowPosition, 20) # cell 높이 설정
            
            
    
    # 감시 데이터 업데이트 / table 에 그리기
    def watch_update(self):
        # 최대 잔고 초과하면 감시 ㄴㄴ
        if(len(self.jango_data) >= self.max_item):
            return
        
        self.tableWidget_watch.setRowCount(0) # 행 갯수 0 으로 초기화
        
        for item in self.obj_watch.dataStg:
            cur_data = cybos_current.data_current(obj_stockMst, item['code'])
            item_list = []
            item_list.append(QTableWidgetItem(item['종목명'])) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(cur_data['cprice'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(cur_data['vol'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(cur_data['vol_value'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(cur_data['offer'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            item_list.append(QTableWidgetItem(format(cur_data['bid'],','))) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            
            # 아이템 텍스트 정렬
            for idx in range(1, len(item_list)):
                item_list[idx].setTextAlignment(Qt.AlignRight | Qt.AlignVCenter)
            
            rowPosition = self.tableWidget_watch.rowCount()
            self.tableWidget_watch.insertRow(rowPosition)
            
            # 아이템 추가
            for idx in range(len(item_list)):
                self.tableWidget_watch.setItem(rowPosition, idx, item_list[idx])
                
            self.tableWidget_watch.setRowHeight(rowPosition, 20) # cell 높이 설정
            
    
    # 내역 업데이트 - 추가
    def history_update(self, history):
        rowPosition = self.tableWidget_history.rowCount()
        self.tableWidget_history.insertRow(rowPosition)
        self.tableWidget_history.setItem(rowPosition , 0, QTableWidgetItem(history[0]))
        self.tableWidget_history.setItem(rowPosition , 1, QTableWidgetItem(str(round(history[1],2))))
        self.tableWidget_history.setItem(rowPosition , 2, QTableWidgetItem(str(history[2])))
        
        if(history[1] > 0):
            color = Qt.red
        else:
            color = Qt.blue
            
        for j in range(self.tableWidget_history.columnCount()):
            try:
                self.tableWidget_history.item(rowPosition, j).setBackground(color)
            except:
                pass
                
    # 자동 매수
    def auto_buy(self):
        for item in self.obj_watch.dataStg:
            # 최대 보유종목 제한 넘어가면 넘김 (예약건 제외하는 것으로 수정)
            if(self.max_item <= len(self.jango_data)):
                #print('> 매수불가 - 최대 잔고량 초과!', len(self.jango_data))
                break
                
            # 이미 보유한 종목이면 넘김
            #jango = list(self.jango_data)
            #print(jango[1][1]['종목코드'])
            if(item['code'] in self.jango_data.keys()):
                #print('> 매수불가 - 이미 보유한 종목!')
                continue
            
            # 지금 거래중인 종목이면 넘김
            miss_keys = self.obj_inquiry.diOrderList.keys()
            if(item['code'] in miss_keys):
                #print('> 매수불가 - 거래중인 종목!')
                continue
            
            # 연속 주문으로 인한 대기를 피하기 위해 바로 탈출
            data = cybos_current.data_current(obj_stockMst, item['code'])
            self.obj_trade.buy(data['code'], 10, data['cprice'])
            break
    
    # 자동 매도
    def auto_sell(self):
        cut_plus = int(self.edit_plus.text())
        cut_minus = -1 * int(self.edit_minus.text())

        for code, value in self.jango_data.items():
            # 잔고수량 없으면 통과
            if(value['매도가능'] == 0):
                continue
            # 익절 3%, 손절 -3% 자동매매. 연속주문 오류 시 처리 추가해야할 듯
            if(value['수익률'] >= cut_plus or value['수익률'] <= cut_minus):
                self.obj_trade.sell(value['종목코드'], value['잔고수량'], value['현재가'])
                now = datetime.datetime.now()
                history = (value['종목명'],value['수익률'],value['평가손익']/1000,('{} : {}'.format(now.hour, now.minute)))
                self.history.append(history)
                self.history_update(history) # 매도 리스트 업데이트
                
    
    # 종료, 마무리, 결과 저장
    def quit(self):
        data = self.history
        m = datetime.datetime.now().month
        d = datetime.datetime.now().day
        file = open('C:\\Users\\김민수\\Desktop\\trading_tmp\\result\\{}_{}_result.csv'.format(m,d),'w')
        writer = csv.writer(file)
        for row in data:
            writer.writerow(row)
            
        file.close()
        self.sell_all()
        
        
    # 안 팔린 종목 전부 일괄매도
    def sell_all(self):
        # 잔고에 있는 모든 종목들 매도
        for data in self.jango_data.items():
            self.obj_trade.sell(data[1]['종목코드'], data[1]['잔고수량'], data[1]['현재가'])

In [ ]:
if __name__ == "__main__":
    
    # 창 닫아도 주피터에서 다시 열수있게 - cmd 실행땐 삭제
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        print('QApplication instance already exists: %s' % str(app))

    window = myWindow()
    window.show()

    app.exec_()
    app.exit()

통신상태 -1 1511 참가신청 후 이용하시기 바랍니다.(모의투자 등록되지 않은 계좌번호)(cii.cif.astbln.selcnsbaseaccpt)


AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

...

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'

AttributeError: 'myWindow' object has no attribute 'jango_data'